<a href="https://colab.research.google.com/github/Chigaga/Dimimmings/blob/master/processing_dimmings_20100801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
#import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import datetime
import dateutil.parser
#from IPython.display import HTML

#py.sign_in("chigaga", "zPIazOZh5yTY1Uqn9z71")



In [23]:
#@title function for the time parsing
#parser function for time
d = dateutil.parser.parse('2010-08-01T07:39:24.620')
d.timestamp()

1280648364.62

In [3]:
tables_folder = Path('/content/drive/My Drive/Phd project/tables_folder')
sectors = pd.read_csv(tables_folder / '20100801.csv')
sectors

,Unnamed: 0,Unnamed: 1,area,lat,lon,sector,brightness,interval,vreme4ko
0,207,0,6.482960e+06,87.187805,-51.018229,1.0,-0.0,0.686664,2010-08-01T07:39:24.620
1,207,1,6.524949e+06,87.085143,-56.789411,1.0,-0.0,0.686664,2010-08-01T07:39:24.620
2,207,2,6.502599e+06,87.267856,-54.488505,1.0,-0.0,0.686664,2010-08-01T07:39:24.620
3,207,3,6.545340e+06,87.147552,-60.276694,1.0,-0.0,0.686664,2010-08-01T07:39:24.620
4,207,4,6.522599e+06,87.337030,-58.161016,1.0,-0.0,0.686664,2010-08-01T07:39:24.620
...,...,...,...,...,...,...,...,...,...
9232794,787,30435,6.356905e+06,89.252626,12.267456,5.0,-0.0,8.063331,2010-08-01T15:02:00.620
9232795,787,30436,6.374428e+06,89.439133,12.321877,5.0,-0.0,8.063331,2010-08-01T15:02:00.620
9232796,787,30437,6.356293e+06,89.371524,31.449367,5.0,-0.0,8.063331,2010-08-01T15:02:00.620
9232797,787,30438,6.392258e+06,89.625862,12.376361,5.0,-0.0,8.063331,2010-08-01T15:02:00.620


In [4]:
legend_list = ['1','2','3','4','5']

In [5]:
#@title forward-backward exp smoothing
def exp_smooth_forward(data, alpha, init=0):
    "Supplementary function for forward exponential smoothing"
    out = np.empty(data.size)
    out[0] = init
    for i in range(1, data.size):
        out[i] = out[i-1] + alpha*(data[i]-out[i-1])
    return out

def exp_smooth_backward(data, alpha, init=0):
    "Supplementary function for backward exponential smoothing"
    out = np.empty(data.size)
    out[-1] = init
    for i in reversed(range(data.size-1)):
        out[i] = out[i+1] + alpha*(data[i]-out[i+1])
    return out
    
def exp_smooth(data, alpha, init=0):
    "Supplementary function for forward then backward exponential smoothing"
    return exp_smooth_backward(exp_smooth_forward(data, alpha, init), alpha, exp_smooth_forward(data, alpha, init)[-1])

In [6]:
#@title function for plotting sector parameters in time
#function to make line plots for 5 sectors
generate_report = False
def plot_traces(sector_list, var, xlable = 'xlable', ylable = 'ylable',
                legend = legend_list, title = 'title', mode='lines' ):

    #plot_races.counter += 1
    #fig_name = 'images/' + str(plot.counter) + '.jpg'
    fig = go.Figure()
    i=0
    for sec in sector_list:
        fig.add_trace(go.Scatter(x=sec['time'], y=sec[var], mode=mode, name = 'sector '+legend_list[i], showlegend = True))
        i+=1
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
       
    if generate_report is True:
        fig.write_image(fig_name)
        display(Image(fig_name))
    else:
        fig.show()
#plot_traces.counter = 0

In [8]:
#@title function time derivative
def time_deriv (sec,param):
    y = sec[param]
    x = sec['time']
    dy = np.zeros(sec[param].shape,np.float)
    dy[0:len(y)-1] = np.diff(y)/np.diff(x)/3600
    dy[-1] = dy[-2]
    return dy

## Preparing data for plotting

Creating new columns in the frame

In [9]:
#create sectors according to the polar angle plot
sectors['sector'] = np.where((sectors.lon > -180) & (sectors.lon <= -130), 1, sectors['sector'])
sectors['sector'] = np.where((sectors.lon > -130) & (sectors.lon <= -80), 2, sectors['sector'])
sectors['sector'] = np.where((sectors.lon > -80) & (sectors.lon <= -30), 3, sectors['sector'])
sectors['sector'] = np.where((sectors.lon > -30) & (sectors.lon <= 20), 4, sectors['sector'])
sectors['sector'] = np.where((sectors.lon > 20) & (sectors.lon <= 70), 5, sectors['sector'])

In [10]:
#time ffom the first dimming appearance
sectors['time'] = sectors['interval']-sectors['interval'].min()
sectors['time'] = np.around(sectors['time'],decimals=3)
sectors['lon_class'] = np.around(sectors['lon'],decimals=-1)
sectors['number'] = 1 #to count pixels for mean brightness

sectors['t'] = sectors.vreme4ko.apply(lambda x: x[11:19]) #time for plotting

##we drop unuseful columns
sectors.rename({"Unnamed: 0":"a","Unnamed: 1":"b" }, axis="columns", inplace=True)
sectors.drop(["a","b","interval","vreme4ko"], axis=1, inplace=True)

#adding distance information
deg_dist = 12.17 #distance of 1 degree in Mm
sectors['ring'] = (90-sectors["lat"]).apply(np.ceil) 
sectors['ring'] = sectors['ring']-sectors['ring'].min()+1
sectors['dist'] = sectors['ring']*deg_dist

#splitting the frame for sectors back again - double number not to mix with distance plot
sec11 = sectors.query('sector == 1.0')
sec22 = sectors.query('sector == 2.0')
sec33 = sectors.query('sector == 3.0')
sec44 = sectors.query('sector == 4.0')
sec55 = sectors.query('sector == 5.0')

#sectors

In [11]:
sectors

,area,lat,lon,sector,brightness,time,lon_class,number,t,ring,dist
0,6.482960e+06,87.187805,-51.018229,3.0,-0.0,0.000,-50.0,1,07:39:24,3.0,36.51
1,6.524949e+06,87.085143,-56.789411,3.0,-0.0,0.000,-60.0,1,07:39:24,3.0,36.51
2,6.502599e+06,87.267856,-54.488505,3.0,-0.0,0.000,-50.0,1,07:39:24,3.0,36.51
3,6.545340e+06,87.147552,-60.276694,3.0,-0.0,0.000,-60.0,1,07:39:24,3.0,36.51
4,6.522599e+06,87.337030,-58.161016,3.0,-0.0,0.000,-60.0,1,07:39:24,3.0,36.51
...,...,...,...,...,...,...,...,...,...,...,...
9232794,6.356905e+06,89.252626,12.267456,4.0,-0.0,7.377,10.0,1,15:02:00,1.0,12.17
9232795,6.374428e+06,89.439133,12.321877,4.0,-0.0,7.377,10.0,1,15:02:00,1.0,12.17
9232796,6.356293e+06,89.371524,31.449367,5.0,-0.0,7.377,30.0,1,15:02:00,1.0,12.17
9232797,6.392258e+06,89.625862,12.376361,4.0,-0.0,7.377,10.0,1,15:02:00,1.0,12.17


preparing a frame grouped by sectors for plotting

In [12]:
#summing all values for finding brightness and area  (ee)
#we put both t and time to have hours and UT time
sum_by_sec = sectors.groupby(['time','t','sector']).sum().reset_index()
sum_by_sec['mean br'] = sum_by_sec['brightness']/sum_by_sec['number']
sum_by_lon = sectors.groupby(['time','t','lon_class']).sum().reset_index()

sum_by_sec_dist = sectors.groupby(['time','t','sector','ring','dist']).size().to_frame('count').reset_index()
#sum_by_lon

## Animation of area/brightness depending on the polar angle

In [13]:
plot_area = sum_by_lon.copy()
plot_area['area']=plot_area['area']/10000000000

fig11 = px.area(plot_area, x='lon_class', y='area',
                  title='<b>Area depending on the polar angle</b>', animation_frame="t",)#, line_close=True)
fig11.update_layout(
    xaxis_title="Longitude [deg]",
    yaxis_title='Area [10^10 km^2]')
fig11.update_layout(xaxis = dict(range=[-180,180]),yaxis = dict(range=[0,3]) )
#fig3.update_xaxes(nticks=5)
fig11.show()

#animation_area_plot_url = py.plot(fig12, filename='animation_area_time', auto_open=False,)
#print(animation_area_plot_url)

In [14]:
plot_br = sum_by_lon.copy()
plot_br['brightness']=plot_br['brightness']/10000
fig12 = px.area(plot_br, x='lon_class', y='brightness',color_discrete_sequence=['rgb(15,8,136)'],
                  title='<b>Intensity decrease depending on the polar angle</b>', animation_frame="t",)#, line_close=True)
fig12.update_layout(
    xaxis_title="Longitude [deg]",
    yaxis_title='Brightness [10^4 DN]')
fig12.update_layout(xaxis = dict(range=[-180,180]),yaxis = dict(range=[0,-15]) )
fig12.show()

#animation_br_plot_url = py.plot(fig11, filename='animation_br_time', auto_open=False,)
#print(animation_br_plot_url)

## Time evolution of area and *brightness*

In [15]:
#@title plot area and its rate evolution
#area in time
area_plot = sum_by_sec.copy()
area_plot['area'] = area_plot['area']/10000000000
area_plot['brightness'] = area_plot['brightness']/10000

sec11 = area_plot.query('sector == 1.0')
sec22 = area_plot.query('sector == 2.0')
sec33 = area_plot.query('sector == 3.0')
sec44 = area_plot.query('sector == 4.0')
sec55 = area_plot.query('sector == 5.0')

fig21 = make_subplots(rows=2, cols=1,shared_xaxes=True,)

fig21.add_trace(go.Scatter(x=sec11['time'], y=exp_smooth(sec11['area'].values,0.4),mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='area 1'), row=1, col=1)
fig21.add_trace(go.Scatter(x=sec22['time'], y=exp_smooth(sec22['area'].values,0.4),mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='area 2'), row=1, col=1)
fig21.add_trace(go.Scatter(x=sec33['time'], y=exp_smooth(sec33['area'].values,0.4),mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='area 3'), row=1, col=1)
fig21.add_trace(go.Scatter(x=sec44['time'], y=exp_smooth(sec44['area'].values,0.4),mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='area 4'), row=1, col=1)
fig21.add_trace(go.Scatter(x=sec55['time'], y=exp_smooth(sec55['area'].values,0.4),mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='area 5'), row=1, col=1)

sm_dy1 = exp_smooth(time_deriv(sec11, 'area'),0.4)*1000
fig21.add_trace(go.Scatter(x=sec11['time'], y=sm_dy1,mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='rate 1'), row=2, col=1)
sm_dy2 = exp_smooth(time_deriv(sec22, 'area'),0.4)*1000
fig21.add_trace(go.Scatter(x=sec22['time'], y=sm_dy2,mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='rate 2'), row=2, col=1)
sm_dy3 = exp_smooth(time_deriv(sec33, 'area'),0.4)*1000
fig21.add_trace(go.Scatter(x=sec33['time'], y=sm_dy3,mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='rate 3'), row=2, col=1)
sm_dy4 = exp_smooth(time_deriv(sec44, 'area'),0.4)*1000
fig21.add_trace(go.Scatter(x=sec44['time'], y=sm_dy4,mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='rate 4'), row=2, col=1)
sm_dy5 = exp_smooth(time_deriv(sec55, 'area'),0.4)*1000
fig21.add_trace(go.Scatter(x=sec55['time'], y=sm_dy5,mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='rate 5'), row=2, col=1)

fig21.update_layout(title='<b>Area and its growth rate for each sector in time</b>',width=800, height=700)
fig21.update_yaxes(title_text="Area [10^10 km^2]", row=1, col=1)
fig21.update_yaxes(title_text="Rate [10^7 km^2/s]", row=2, col=1)
fig21.update_xaxes(nticks=10,title_text="Time [h]")
fig21.show()

#area_plot_url = py.plot(fig21, filename='area', auto_open=False,)
#print(area_plot_url)

In [16]:
#@title brightness evolution
#br in time

fig22 = make_subplots(rows=1, cols=1,shared_xaxes=True,)

fig22.add_trace(go.Scatter(x=sec11['time'], y=exp_smooth(sec11['brightness'].values,0.4),mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='sector 1'), row=1, col=1)
fig22.add_trace(go.Scatter(x=sec22['time'], y=exp_smooth(sec22['brightness'].values,0.4),mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='sector 2'), row=1, col=1)
fig22.add_trace(go.Scatter(x=sec33['time'], y=exp_smooth(sec33['brightness'].values,0.4), line = dict(color='#00CC96'),legendgroup="group3",name='sector 3'), row=1, col=1)
fig22.add_trace(go.Scatter(x=sec44['time'], y=exp_smooth(sec44['brightness'].values,0.4), line = dict(color='#AB63FA'),legendgroup="group4",name='sector 4'), row=1, col=1)
fig22.add_trace(go.Scatter(x=sec55['time'], y=exp_smooth(sec55['brightness'].values,0.4), line = dict(color='#FFA15A'),legendgroup="group5",name='sector 5'), row=1, col=1)

#sm_dy1 = exp_smooth(time_deriv(sec11, 'brightness'),0.4)*10
#fig.add_trace(go.Scatter(x=sec11['time'], y=sm_dy1,mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='rate 1'), row=2, col=1)
#sm_dy2 = exp_smooth(time_deriv(sec22, 'brightness'),0.4)*10
#fig.add_trace(go.Scatter(x=sec22['time'], y=sm_dy2,mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='rate 2'), row=2, col=1)
#sm_dy3 = exp_smooth(time_deriv(sec33, 'brightness'),0.4)*10
#fig.add_trace(go.Scatter(x=sec33['time'], y=sm_dy3,mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='rate 3'), row=2, col=1)
#sm_dy4 = exp_smooth(time_deriv(sec44, 'brightness'),0.4)*10
#fig.add_trace(go.Scatter(x=sec44['time'], y=sm_dy4,mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='rate 4'), row=2, col=1)
#sm_dy5 = exp_smooth(time_deriv(sec55, 'brightness'),0.4)*10
#fig.add_trace(go.Scatter(x=sec55['time'], y=sm_dy5,mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='rate 5'), row=2, col=1)

fig22.update_layout(title='<b>Brightness evolution for each sector</b>',width=800, height=400)
fig22.update_yaxes(title_text="Brightness [10^4 DN]", row=1, col=1)
fig22.update_yaxes(title_text="Rate [10^3 DN/s]", row=2, col=1)
fig22.update_xaxes(nticks=10,title_text="Time [h]")
fig22.show()

#br_plot_url = py.plot(fig22, filename='br', auto_open=False,)
#print(br_plot_url)

In [17]:
#@title mean brightness evolution

fig23 = make_subplots(rows=1, cols=1,shared_xaxes=True,)

fig23.add_trace(go.Scatter(x=sec11['time'], y=exp_smooth(sec11['mean br'].values,0.4),mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='sector 1'), row=1, col=1)
fig23.add_trace(go.Scatter(x=sec22['time'], y=exp_smooth(sec22['mean br'].values,0.4), line = dict(color='#EF553B'),legendgroup="group2",name='sector 2'), row=1, col=1)
fig23.add_trace(go.Scatter(x=sec33['time'], y=exp_smooth(sec33['mean br'].values,0.4), line = dict(color='#00CC96'),legendgroup="group3",name='sector 3'), row=1, col=1)
fig23.add_trace(go.Scatter(x=sec44['time'], y=exp_smooth(sec44['mean br'].values,0.4), line = dict(color='#AB63FA'),legendgroup="group4",name='sector 4'), row=1, col=1)
fig23.add_trace(go.Scatter(x=sec55['time'], y=exp_smooth(sec55['mean br'].values,0.4),mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='sector 5'), row=1, col=1)

fig23.update_layout(title='<b>Mean brightness evolution in time evolution for each sector</b>',width=800, height=400)
fig23.update_yaxes(title_text="Mean brightness [DN]", row=1, col=1)
fig23.update_xaxes(nticks=10,title_text="Time [h]")
fig23.show()

#mean_br_plot_url = py.plot(fig23, filename='mean_br', auto_open=False,)
#print(mean_br_plot_url)

In [ ]:
#@title (old) fun plot derivative
#mode: 'edge' or 'area' - to plot different parameters
def plotting_edge_rate(sec, mode, color,group,name):
  rr = sec
  x = rr['time']
  if mode=='edge':
    y = rr['max']
  else:
    y = rr['area']

  dy = np.zeros(y.shape,np.float)
  dy[0:len(y)-1] = np.diff(y)/np.diff(x)/3600
  dy[-1] = dy[-2]

  return (fig.add_trace(go.Scatter(x=x, y=dy,mode='lines',line = dict(color=color),legendgroup=group,name=name), row=2, col=1))

In [18]:
#@title Smoothing of the area rate comparison
dy2 = time_deriv(sec11, 'area')*1000
sm_dy2 = exp_smooth(time_deriv(sec11, 'area'),0.4)*1000
fig = go.Figure()
fig.add_trace(go.Scatter(x=sec22['time'], y=dy2, mode='lines', name = 'rate', showlegend = True))
fig.add_trace(go.Scatter(x=sec22['time'], y=sm_dy2, mode='lines',name = 'smoothed rate', showlegend = True))

fig.update_layout(title='<b>Smoothing of the area growth rate for sector 1</b>')
fig.update_yaxes(title_text="Rate [10^7 km^2/s]")
fig.update_xaxes(nticks=10,title_text="Time [h]")
fig.show()

In [ ]:
#to plot just time evolution of areas
sector_list = [sec11,sec22,sec33,sec44,sec55]
plot_traces(sector_list, var='area', xlable = 'Time [h]', ylable = 'Area [10^10 km^2]',
                legend = legend_list, title = 'Area evolution for each sector', mode='lines' )

# Leading edge analysis

Figure 0: distribution of pixels by distance

In [43]:
#@title to plot dist at 1 time step
##if we want to check one time step
t = 1.09
time1 = sectors[sectors['time'] == t]
rr = time1
#sec_string = str(rr['sector'][0])
one_step = rr.groupby(['time','sector','dist']).size().to_frame('count').reset_index()
fig0 = px.line(one_step, x='dist', y='count', color='sector',#line_group="sector",range_x=[-180,180],
                title='<b>Distance of pixels for each sector after 1 hour from the dimming appearance </b>')#, line_close=True)
fig0.update_layout( xaxis_title="Distance from the center [Mm]",  yaxis_title='count')
fig0.update_xaxes(range=[0,500])
fig0.show()

one_time_plot_url = py.plot(fig0, filename='one_time', auto_open=False,)
print(one_time_plot_url)

NameError: ignored

Figure 1: pixels by distance for all time steps

In [ ]:
#@title all distances distribution 
fig1 = px.line(sum_by_sec_dist, x='dist', y='count', color='sector',line_group="time",# range_x=[-180,180],
            title='<b>Distance of pixels for each sector </b>'  )#, line_close=True)
fig1.update_layout(xaxis_title="Distance from the center [Mm]",   yaxis_title='count')
fig1.update_xaxes(range=[0,400])
fig1.show()


Now we want to create a minimal number of pixels in each ring to be counted as leading edge: we take 10% of each ring's area. 

Figure 2: time profile for the leading edge

In [45]:
#area of the ring  - we know that radius of each ring is 6 pixels
percent = 0.05 #percent of area enough for the lead edge
sum_by_sec_dist['S'] = (2*sum_by_sec_dist['ring']*6-1)*math.pi*percent

sum_by_sec_dist['edge'] = np.where((sum_by_sec_dist['count'] >= sum_by_sec_dist['S']),sum_by_sec_dist['dist'], np.nan)
sum_by_sec_dist = sum_by_sec_dist.dropna()
distance=sum_by_sec_dist.groupby(['time','sector']).edge.agg([max]).reset_index()
#distance=sum_by_sec_dist.groupby(['time','sector'])['edge'].transform(max).reset_index()
sum_by_sec_dist

,time,t,sector,ring,dist,count,S,edge
1,0.000,07:39:24,1.0,3.0,36.51,30,5.497787,36.51
2,0.000,07:39:24,1.0,4.0,48.68,11,7.382743,48.68
3,0.000,07:39:24,1.0,5.0,60.85,33,9.267698,60.85
4,0.000,07:39:24,1.0,6.0,73.02,82,11.152654,73.02
5,0.000,07:39:24,1.0,7.0,85.19,61,13.037610,85.19
...,...,...,...,...,...,...,...,...
55699,7.377,15:02:00,5.0,14.0,170.38,59,26.232299,170.38
55702,7.377,15:02:00,5.0,24.0,292.08,73,45.081855,292.08
55703,7.377,15:02:00,5.0,25.0,304.25,120,46.966810,304.25
55704,7.377,15:02:00,5.0,26.0,316.42,91,48.851766,316.42


function to plot the time derivative

In [46]:
#@title Time evolution of distance and its rate
sec1 = distance.query('sector == 1.0')
sec2 = distance.query('sector == 2.0')
sec3 = distance.query('sector == 3.0')
sec4 = distance.query('sector == 4.0')
sec5 = distance.query('sector == 5.0')

fig4 = make_subplots(rows=2, cols=1,shared_xaxes=True,)
fig4.add_trace(go.Scatter(x=sec1['time'], y=sec1['max'],mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='edge 1'), row=1, col=1)
fig4.add_trace(go.Scatter(x=sec2['time'], y=sec2['max'],mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='edge 2'), row=1, col=1)
fig4.add_trace(go.Scatter(x=sec3['time'], y=sec3['max'],mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='edge 3'), row=1, col=1)
fig4.add_trace(go.Scatter(x=sec4['time'], y=sec4['max'],mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='edge 4'), row=1, col=1)
fig4.add_trace(go.Scatter(x=sec5['time'], y=sec5['max'],mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='edge 5'), row=1, col=1)

dy1_dist = time_deriv(sec1, 'max')
fig4.add_trace(go.Scatter(x=sec1['time'], y=dy1_dist,mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='rate 1'), row=2, col=1)
dy2_dist = time_deriv(sec2, 'max')
fig4.add_trace(go.Scatter(x=sec2['time'], y=dy2_dist,mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='rate 2'), row=2, col=1)
dy3_dist = time_deriv(sec3, 'max')
fig4.add_trace(go.Scatter(x=sec3['time'], y=dy3_dist,mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='rate 3'), row=2, col=1)
dy4_dist = time_deriv(sec4, 'max')
fig4.add_trace(go.Scatter(x=sec4['time'], y=dy4_dist,mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='rate 4'), row=2, col=1)
dy5_dist = time_deriv(sec5, 'max')
fig4.add_trace(go.Scatter(x=sec5['time'], y=dy5_dist,mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='rate 5'), row=2, col=1)

fig4.update_layout(title='<b>Leading edge and its rate for each sector in time</b>',width=800, height=700)
fig4.update_yaxes(title_text="Leading edge [Mm]", row=1, col=1)
fig4.update_yaxes(title_text="Rate [Mm/s]", row=2, col=1)
fig4.update_xaxes(nticks=10,title_text="Time [h]")
fig4.show()

#lead_edge_plot_url = py.plot(fig4, filename='lead_edge', auto_open=False,)
#print(lead_edge_plot_url)

# Animation plots with sectors on x-axis


In [ ]:
#@title animated plot for area
b = area_plot
fig31 = px.area(b, x='sector', y='area', color='time',
                   animation_frame="time",
                    color_discrete_sequence=['rgb(15,8,136)'],  
                  title='<b>Animation of the instantaneous area</b>')
fig31.update_layout(yaxis_title='Area [10^10 km^2]')                   
fig31.update_layout(xaxis = dict(range=[1,5]),yaxis = dict(range=[0,5]) ,showlegend=False )
fig31.update_xaxes(nticks=5)
fig31.show()
#anim_area_plot_url = py.plot(fig31, filename='anim_area', auto_open=False,)
#print(anim_area_plot_url)

In [ ]:
#@title animation of brightness
b = area_plot
fig32 = px.area(b, x='sector', y='brightness', color='time',
                   animation_frame="time",
                 
                    color_discrete_sequence=['rgb(198,66,124)']   
                  ,title='<b>Animation of the instantaneous brightness</b>')
                   
fig32.update_layout(xaxis = dict(range=[1,5]),yaxis = dict(range=[0,-10]),yaxis_title='Brightness [10^4 DN]',showlegend=False )
fig32.update_xaxes(nticks=5)
fig32.show()
#anim_br_plot_url = py.plot(fig32, filename='anim_br', auto_open=False,)
#print(anim_br_plot_url)

In [ ]:
#@title animation of mean br
b = sum_by_sec
fig33 = px.area(b, x='sector', y='mean br', color='time',
                   animation_frame="time",
                    #hover_name="number of pixels"
                      # , color_discrete_sequence=px.colors.sequential.Plasma
                    color_discrete_sequence=['rgb(210,80,112)']   
                  ,title='<b>Animation of the mean brightness</b>')
                   
fig33.update_layout(xaxis = dict(range=[1,5]),yaxis = dict(range=[0,-50]),yaxis_title='Mean brightness [DN]',showlegend=False  )
fig33.update_xaxes(nticks=5)
fig33.show()

#anim_mean_br_plot_url = py.plot(fig33, filename='anim_mean_br', auto_open=False,)
#print(anim_mean_br_plot_url)

In [ ]:
#@title Creating variables for text in html
# code for creating the html page

first_head = '''
<html>
    <head>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
        <style>body{ margin:0 100; background:whitesmoke; }</style>
    </head>
    <body>
        <h1>Analysis of the dimming event 2011 10 01</h1>

        <!-- *** Section 1 *** --->
        <h2>Section 1: Animation of the area/brightness depending on the polar angle</h2>
          <p> The disrtribution of pixels and their intensity by the polar angle can be the first step of the dimming direction analysis, 
        helping to define the sectors. We sort the area/intensity values by the longitude, sum the values in a certin range (10 degrees) and plot as a function of angle. 
        By the slider you can choose a certain time step or speed up the animation.</p>
'''
  
first_first = '''
<p>Figure 1.1 shows the continuous distribution of pixels by the polar angle. The animation shows the smoothed by 10 degrees curves of the area depending on angle. 
        The time is presented in hours from the first dimming appearance. 
        The dimming is growing mostly between [-100, 50] degrees. </p>
'''
first_second = '''
 <p>  Figure 1.2 presents the dimming intensity depending on the angle (smoothed by 10 degrees). 
 
        Altough the area is growing between -100 and 50 degrees, at the beginning the part [-70, -30] has the biggest intensity, 
        while after 2 hours from the dimming appearance the part [-30, 50] is darkening.  </p>
        '''

second_head = '''        
              <!-- *** Section 2 *** --->

        <h2>Section 2: Time evolution plots</h2>
        <p> We separated the image on 5 angular sectors by 30 degrees with the center in the eruption source. 
        Each sector is presented by its own color. Sequence of dimming maps with time cadence 12 sec were used.
        A certain time range can be selected to scale the plot.
        All the curves are smoothed using forward-backward exponential smoothing method. </p> 
'''
second_first = '''
<p> Figure 2.1 shows the area (top panel) and its growth rate (bottom panel) evolution in time for each sector. 
        The area and growth rate parameters are grouped for each sector by a separate color and can be highlighted individually.
        Time is shown in hours from the first dimming appearance. </p>

       <p>  Sectors 2 and 3 seem to present the dominant direction, 
        having a rapid growth in the first hour from the dimming appearance and leading by the numer of pixels. </p>

'''
second_second = '''
<p> Figure 2.2 shows the brightness evolution in time for each sector. The brightness is presented in 10^4 DN.
        Time is shown in hours from the first dimming appearance. </p>

<p>We can notice that sector 2 is significantly dominating by the intensity and has a rapid decrease 
after 3 hours of evolution. The second darkest sector is sector 1, 
despite the face that sector 3 has more pixels, but it reaches its minimum after the sector 3.</p>
'''
second_third = '''
<p> Figure 2.3 shows the mean brightness evolution in time for each sector.
        Time is shown in hours from the first dimming appearance. </p>

        <p>Sector 1 has the darkest pixels, slightly decreasing its intensity during the time series. 
        Sectors 1 and 2, 3 and 4 show similar level and evolution of the mean intensity. </p>
'''
third_head = '''
<h2>Section 3: Animations of the parameters depending on the sector</h2>
        <p> Animations present the distribution of area/brightness/mean brightness for every sector in time. 
        The time is presented on the slider in hours from the first dimming appearance. Every certain time step 
        can be chosen by moving the slider.</p> 
'''
third = '''<p> </p> '''

fourth_head = '''
<h2>Section 4: Leading edge </h2>
        <p>We divide the image into a number of rings of equal width around the eruptive center. 
        Each ring represents the area of points at the same distance from the center.  
        We count the number of pixels in each ring for each sector and analyze it for the different time steps. 
        Then we transform the ring number to the actual distance in Mm, 
        and relate the outer border of every ring to the corresponding distance from the eruptive center.  </p> 
'''

fourth = '''
<p> Figure 4 shows the leading edge (top panel) and its change rate (bottom panel) evolution in time for all sectors.
The area and growth rate parameters are grouped for each sector by a separate color and can be highlighted individually.
        Time is shown in hours from the first dimming appearance.  </p>

    </body>
</html>'''


In [ ]:
#@title Combinig all the materials in one html file
#Finally, write the html string to a local file.

f = open(tables_folder / 'newnew.html','w')
f.write(first_head)
f.write(fig11.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(first_first)
f.write(fig12.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(first_second)
f.write(second_head)
f.write(fig21.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(second_first)
f.write(fig22.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(second_second)
f.write(fig23.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(second_third)
f.write(third_head)
f.write(fig31.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(fig32.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(fig33.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(third)
f.write(fourth_head)
f.write(fig4.to_html(full_html=False, include_plotlyjs='cdn', default_height = 700, default_width = 1200))
f.write(fourth)
f.close()